In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
def savetoShapefile():
    cap_dict = {}
    with open('TRTS4S_Capacity.csv', 'r') as f:
        lines = f.readlines()
        for line in lines[1:]:
            parts = line.strip().split(',')
            class_num = int(parts[0])
            for i in range(1, len(parts)):
                lanes = i
                capacity = int(parts[i])
                cap_dict[(class_num, lanes)] = capacity
    ue_results = pd.read_csv(f"{year}_{scenario}_{rush_hour}{case}_UE_results.dat", sep="\t")
    gdf = gpd.read_file(f"Network/Net_modified_{scenario}_{rush_hour}調撥.shp")
    ue_results.rename(columns={"tailNode": "A", "headNode": "B"}, inplace=True)
    gdf = gdf.merge(ue_results[["A", "B", "capacity", "UE_travelTime", "UE_flow"]], on=["A", "B"], how="left")
    gdf['capacity'] = gdf.apply(lambda row: get_capacity(row, cap_dict), axis=1)
    gdf["V/C"] = gdf["UE_flow"] / gdf["capacity"]
    gdf["assigned_speed"] = gdf["LENGTH"] / gdf["UE_travelTime"] * 60 
    gdf.to_file(f"{year}_{scenario}_{rush_hour}{case}_UE_results.shp")

def get_capacity(row, cap_dict):
    class_num = row['CLASS_110']
    lanes = row['LANES_110']
    if (class_num, lanes) in cap_dict:
        return cap_dict[(class_num, lanes)]
    else:
        print(f"Warning: No capacity found for class {class_num} and lanes {lanes}. Using default value 0.")
        return 0

year = 104
for scenario in ['現況']: # '現況', '全線拆除', '部分拆除', '', ''
    for rush_hour in ['晨峰']: # '晨峰', '昏峰'
        for case in ['']: # '', '士林北投', '士林北投三重', '全區域'
            savetoShapefile()

C:\Users\chiuj\AppData\Local\Temp\ipykernel_3308\3298499467.py:19: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(f"{year}_{scenario}_{rush_hour}{case}_UE_results.shp")
C:\Users\chiuj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
C:\Users\chiuj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'UE_travelTime' to 'UE_travelT'
  ogr_write(
C:\Users\chiuj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered